<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>CS 144 - Winter 2024 - Mini-Assembler</h1></center>
<center><h1>Due: Sunday, March 17th, 2024 @ 11:59pm</h1></center>

### Enter your information below:

<div style="color: #000000;background-color: #EEEEFF">
    Your Name (submitter): Isaiah Bernardino <br>
    Your student ID (submitter): 862266310
<br>
<br>
<b>By submitting this notebook, I assert that the work below is my own work, completed for this course.  Except where explicitly cited, none of the portions of this notebook are duplicated from anyone else's work or my own previous work.</b>
<br>    
<br>
<b>Instruction for submissions:</B> when you have completed this project, download this .ipynb file to your computer by left-clicking on the file name, and submit to <a href="https://elearn.ucr.edu/">Canvas</A> by the deadline. 
<br>
<br>
<B>Late work:</B> There is no late deadline for the final project, except for the most serious circumstances (illness, medical emergency, etc.) which have to documented.
</div>


<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Assembler</h1></center>
<br>


The objective of this project is to implement software to assemble a (very short and non-repetitive) genome from synthetic reads. Here we assume that reads have no sequencing errors. (Credit: Ben Langmead)


**Step 1:** Let's get the reads in FASTA format. The reads come from the same synthetic genome and each is 100 bp long. For simplicity, these reads don't have any quality values. 

In [ ]:
!wget http://www.cs.ucr.edu/~stelo/cs144winter24/data/reads.fa

: 

**Step 2:** Write code to find each read's *best buddy to the right* with a time-efficient algorithm. For each read **A**, find the other read **B** that has the **longest** suffix/prefix exact match with **A**. In other words, if a suffix of **A** matches a prefix of **B**, **B** is **A**'s *best buddy* to the right. **However**, if there is a tie, or if the longest suffix/prefix match is less than 40 nucleotides long, then **A** *has no best buddy to the right*.  For each read, your program should output either (a) nothing, if there is no best buddy to the right, or (b) a single, space-separated line with the IDs of **A** and **B** and the length of the overlap, like this:

    0255/2 2065/1 88

This indicates a 88 bp suffix of the read with ID **0255/2** that matches a prefix of the read with ID **2065/1**.  Because of how we defined best buddy, it also means no other read besides **2065/1** has a prefix of 88bp or longer that is also a suffix of read **0255/2**.  A corrolary of this is that a particular read ID should appear in the first column of your program's output at most once.  Also, since we require the overlap to be at least 40 bases long, the last column should not contain any number smaller than 40.

Notes:
* You can assume all reads are error-free and from the forward strand.  In other words, you do **not** need to consider sequencing errors or reverse complements
* The order of the output lines is not important
* Reading/parsing the FASTA files can be done with BioPython, or by borrowing code from <A HREF="https://nbviewer.jupyter.org/github/BenLangmead/comp-genomics-class/blob/master/notebooks/FASTA.ipynb">this</A> if you want your own custom solution

Important: Try to avoid comparing every read to every other read when looking for suffix/prefix matches. This can be achieved by grouping "similar" reads, that is reads that share common k-mers.  For instance, you can build a dictionary where the keys are k-mers and the values are sets containing the names of all reads containing that k-mer.  Since you are looking for overlaps of length at least 40, you only need to compare reads if they have at least 1 40-mer in common.

You are allowed to use any Python package that you want. You should be able install packages using `!pip install package`. Remember that it is mandatory to acknowledge sources.

In [8]:
## YOUR CODE HERE
## you are allowed to use any python library you want
from Bio import SeqIO
def best_buddy(R):
    temp_mer = {}
    match = {}
    k_mer = {}
    ties = []
    min_buddy = 40
    n = 100
    SeqDict = SeqIO.to_dict(SeqIO.parse(R, "fasta"))

    ## 40-mer Dictionary
    for seq_record in SeqIO.parse(R, "fasta"):
        for i in range(len(seq_record.seq)-min_buddy+1):
            if str(seq_record.seq[i:i+min_buddy]) in temp_mer.keys():
                temp_mer[str(seq_record.seq[i:i+min_buddy])].append(seq_record.name)
            else:
                temp_mer[str(seq_record.seq[i:i+min_buddy])] = [seq_record.name]

    
    for j in range(n-1, 39, -1):
        count = 0
        temp_keys = list(temp_mer.keys())
        for word in temp_keys:
            for suffix in temp_mer[word]:
                    for i in range(len(seq_record.seq)-j+1):
                        if SeqDict[suffix].seq[i:i+j] in k_mer.keys():
                            k_mer[str(SeqDict[suffix].seq[i:i+j])].add(suffix)
                        else:
                            k_mer[str(SeqDict[suffix].seq[i:i+j])] = {suffix}

        k_mer_keys = list(k_mer.keys())
        for k in k_mer_keys: 
            for idx,suffix in enumerate(k_mer[k]):
                for idx2,prefix in enumerate(k_mer[k]):
                    if idx2!=idx and SeqDict[suffix].seq[n-j:n] == k and SeqDict[prefix].seq[0:j] == k:
                        if suffix in match.keys():
                            if j == match[suffix][1]:
                                match[suffix][2] = "eq"
                        else:
                            match[suffix] = [prefix, j,"neq"]
                            count += 1
        temp_mer = k_mer
        k_mer = {}
        if count == 0: break   
                        
    match_keys = list(match.keys())
    match_keys.sort()

    for A in match_keys:
        if match[A][2] == "eq": match.pop(A)
            
    return match

def print_bbr(bbr):
    bbr_keys = list(bbr.keys())
    bbr_keys.sort()

    for A in bbr_keys:
        print(A, bbr[A][0], bbr[A][1])

In [16]:
%%time
bbr = best_buddy(r"reads.fa")
# print_bbr(bbr)

CPU times: user 20.2 s, sys: 87.1 ms, total: 20.3 s
Wall time: 20.3 s


**Step 3:** Write a program that takes the output of the overlap program from Step 2 and creates **uni**quely assemblable con**tigs** (unitigs), using the algorithm described below

We already determined each read's best buddy *to the right*.  I'll abbreviate this as bbr.  We did not attempt to compute each read's best buddy *to the left* (bbl), but we can infer it from the bbrs.  Consider the following output:

    A B 60
    E A 40
    C B 70
    D C 40

**A**'s bbr is **B**.  But **B**'s bbl is **C**, *not* **A**!  Your program should form unitigs by joining together two reads **X** and **Y** if they are *mutual best buddies*.  **X** and **Y** are mutual best buddies if **X**'s bbr is **Y** **and** **Y**'s bbl is **X**, or vice versa.  In this example, we would join **D**, **C**, and **B** into a single unitig (and in that order), and would join reads **E** and **A** into a single unitig (also in that order).

Your program's output should consist of several entries like the following, with one entry per unitig:

    START UNITIG 1 D
      C 40
      B 70
    END UNITIG 1
    START UNITIG 2 E
      A 40
    END UNITIG 2

The first entry represents a unitig with ID **1** consisting of 3 reads.  The first (leftmost) read is **D**.  The second read, **C**, has a **40** nt prefix that is a suffix of the previous read (**D**).  The third (rightmost) read in the contig (**B**) has a **70** bp prefix that is a suffix of the previous read (**C**).

Each read should be contained in exactly one unitig.  The order of unitigs in the file is not important, but the unitig IDs should be integers and assigned in ascending order.

You are allowed to use any Python package that you want but it is mandatory to acknowledge sources.

**Hint:** the correct solution for this particular input consists of exactly 4 unitigs

In [12]:
## YOUR CODE HERE
## you are allowed to use any python library you want
import networkx as nx
def unitigs(mutual_b):     
    open("unitig.txt", "w").close()
    bbr_keys = list(mutual_b.keys())
    bbr_keys.sort()
    
    for idx1,A in enumerate(bbr_keys):
        for idx2,B in enumerate(bbr_keys):
            if idx1 != idx2 and mutual_b[A][0] == mutual_b[B][0]:
                if mutual_b[A][1] > mutual_b[B][1]:
                    mutual_b[B][2] = "del"
                elif mutual_b[A][1] < mutual_b[B][1]:
                    mutual_b[A][2] = "del"
                else:
                    mutual_b[A][2] = mutual_b[A][2]= "del"
            
    for A in bbr_keys:
        if mutual_b[A][2] == "del": mutual_b.pop(A)

    bbr_keys = list(mutual_b.keys())
    suffix_list = [mutual_b[k][0] for k in bbr_keys]
    suffix_not =[k for k in bbr_keys if not k in suffix_list] 
    
    for i,suffix in enumerate(suffix_not):
        g = nx.DiGraph()
        g.add_node(suffix)
        g.add_node(mutual_b[suffix][0])
        g.add_edge(suffix, mutual_b[suffix][0], weight=mutual_b[suffix][1])
        nextKey = mutual_b[suffix][0]

        while nextKey in bbr_keys:
            g.add_node(mutual_b[nextKey][0])
            g.add_edge(nextKey, mutual_b[nextKey][0], weight=mutual_b[nextKey][1])
            nextKey = mutual_b[nextKey][0]

        nodes = list(g.nodes)
        edges = list(g.edges.data("weight", default=1))
        
        with open('unitig.txt', 'a') as f:
            f.write('START UNITIG '+ str(i) + ' ' + str(nodes[0]) + '\n')
            for j,node in enumerate(nodes):
                if j != 0:
                    f.write(str(node) + ' ' + str(edges[j-1][2]) + '\n')
            f.write('END UNITIG'+ ' ' + str(i) + '\n')
        g.clear()
    return
            

In [13]:
%%time
unitigs(bbr)

CPU times: user 6.37 s, sys: 6.03 ms, total: 6.37 s
Wall time: 6.37 s


**Step 4**: Write a program that produces the final genome by processing the unitig file. This requires that you compare the unitigs to each other, think about what order they must go in, and then put them together accordingly. The final genome should be written in FASTA format (again, use BioPython or write your own code). You are allowed to use any Python package that you want, but it is mandatory to acknowledge sources.

**Hints:**

* the correct genome is 7959 nucleotides long
* to learn how the unitigs should go together, try overlapping them with each other
* a unitig can be used more than once in the solution

In [14]:
## YOUR CODE HERE
## you are allowed to use any python library you want
def asm_genome(unitig):
    asm_unitig = []
    SeqDict = SeqIO.to_dict(SeqIO.parse(r"reads.fa", "fasta"))

    for j in range(len(unitig)):
        for i in range(len(unitig[j])):
            if i == 0:
                x = str(SeqDict[unitig[j][i]].seq)
            else:
                w = unitig[j][i].split()
                x += str(SeqDict[w[0]].seq)[int(w[1]):100]
        asm_unitig.append(x)
    
    suffix_overlap = []
    prefix_overlap = []
    
    for k in range(len(asm_unitig)):
        dict_suffix = {}
        dict_prefix = {}
        for i in range(len(asm_unitig)):
            if i != k:
                j = asm_unitig[i].find(asm_unitig[k][0])
                while j >= 0:
                    if asm_unitig[k].startswith(asm_unitig[i][j:]) and len(asm_unitig[i][j:]) > 39:
                        dict_suffix[i] = j
                        break
                    j = asm_unitig[i].find(asm_unitig[k][0], j+1)
                    
                j = asm_unitig[k].find(asm_unitig[i][0])
                while j >= 0:
                    if asm_unitig[i].startswith(asm_unitig[k][j:]) and len(asm_unitig[k][j:]) > 39:
                        dict_prefix[i] = j
                        break
                    j = asm_unitig[k].find(asm_unitig[i][0], j+1)
        suffix_overlap.append(dict_suffix)
        prefix_overlap.append(dict_prefix)
        
    current = -1
    end = -1
    for i in range(len(suffix_overlap)):
        if len(suffix_overlap[i]) == 0: current = i
            
    for i in range(len(prefix_overlap)):
        if len(prefix_overlap[i]) == 0: end = i
            
    visit_all = [0]*len(asm_unitig)
    genome = asm_unitig[current]
    while sum(visit_all) != len(asm_unitig):
        visit_all[current] = 1
        next = prefix_overlap[current]
        
        if sum(visit_all) == len(asm_unitig)-1 and end in list(next.keys()):
            visit_all[end] = 1
            key = end
        else:   
            ok = False
            if len(list(next.keys())) == 1: key = list(next.keys())[0]
            else:
                for key in list(next.keys()):
                    for i in range(len(suffix_overlap)):
                        if key in list(suffix_overlap[i].keys()) and key != end:
                            ok = True
                            break
                    if ok: break
        
        genome = genome + asm_unitig[key][len(asm_unitig[current])-next[key]:]
        current = key

    records = ">" + str(len(genome)) + "\n" + genome
    with open("rg.fa", "w") as handle:
      handle.write(records)


In [15]:
%%time
unitigs(best_buddy(r"reads.fa"))
unitig = []

with open('unitig.txt', "r") as f:
    for line in f:
        check = line.rstrip().split()[0]
        if check == 'START':
            _ = []
            _.append(line.rstrip().split()[3])
        elif check != 'END':
            _.append(line.rstrip())
        elif check == 'END':
            unitig.append(_)

asm_genome(unitig)

CPU times: user 26.9 s, sys: 74.9 ms, total: 27 s
Wall time: 27.3 s
